# FIT5196 Task 3 in Assessment 1
#### Student Name: Boyu Zhang
#### Student ID: 28491300

Date: XX/XX/XXXX

Version: 1.0

Environment: Python 3.6 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.:
* re (for regular expression, included in Anaconda Python 3.6) 
* json (for import the data into json format file, included in Anaconda Python 3.6) 


## 1.  Import libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import re
import json

## 2. Parse XML File

In this section, you can write your python scripts to parse the correspondiing file.
You should 
* write proper notes for all code block in this notebook using the Markdown cells
* provide proper comment in your scripts
* run all cells to make sure scripts are runable. If the scripts cannot be run by the assessors, they will not be assessed and zero mark will be given to the task.

### Load and explore source data

In [2]:
#Initialize the outest structure which is a python dictionary
sport = {}
sport['thesaurus'] = []

In [3]:
#choose data structure to contain the basic object in which case is each Term
sport

{'thesaurus': []}

In [4]:
# open the xml file and observer the patter of strings
xml_file = './Input/australian-sport-thesaurus-student.xml'
with open(xml_file) as f:
    lines = f.readlines()
    print(lines[2:11])

['  <Term>\n', '    <Title>.177 (4.5mm) Airgun</Title>\n', '    <Description>The standard airgun calibre for international target shooting.</Description>\n', '    <RelatedTerms>\n', '      <Term>\n', '        <Title>Shooting sport equipment</Title>\n', '        <Relationship>Narrower Term</Relationship>\n', '      </Term>\n', '    </RelatedTerms>\n']


### Find approach to extract elements and put them in proper data structure

In [5]:
#devise regex patterns to match the boundary of each term object
term_start = '\s{2}<Term>\n'
term_end = '\s{2}</Term>\n'
#devise a regex patter to capture the content of each element e.g 'Title','Description'
p_element = '^\s{4}<(\w+)>(.+)</\w+>'
#design a regex patter to capture the 'Related terms' element which is spread in multi-lines
relation_start = '\s{4}<RelatedTerms>\n'
relation_end = '\s{4}</RelatedTerms>\n'
# a regex pattern to identify the boundaries of each object in the RelatedTerms list
inner_start = '\s{6}<Term>\n'
inner_end = '\s{6}</Term>\n'

In [6]:

#use a list to retain the strings of first term
bounds  = []
with open(xml_file) as f:
    lines = f.readlines()
    for i,line in enumerate(lines):
        if re.match(term_start,line):
            bound = []
            bound.append(i)
        if re.match(relation_start,line):
            bound.append(i)
            inner_bound = []
        if re.match(inner_start,line):
            inner_bound.append(i)
        if re.match(inner_end,line):
            inner_bound.append(i)
        if re.match(relation_end,line):
            bound.append(i)
            bound.append(inner_bound)
            #inner_bounds.append(inner_bound)
        if re.match(term_end,line):
            bound.append(i)
            bounds.append(bound)
   

In [12]:
#devise a regex pattern to 'Title' and 'Relationship' element of the objects in the
#RelatedTerms list
r_element = '\s{8}<(\w+)>(.+)</\w+>'

In [13]:
#the full process:
for bound in bounds:
    term = {}
    related_list = []
    # capture the elements of each outer object
    outer = lines[bound[0]:bound[1]]
    for line in outer:
        m = re.search(p_element,line)
        try:
            term[m.group(1)] = m.group(2)
        except:
            continue
    
    #skip the term if it has no related terms
    if len(bound) == 2:
        continue
    inner = bound[3]
    #capture element for objects inside the RelatedTerms list
    for i in range(0,len(bound[3])-1,2):
        inner_term = {}
        for line in lines[inner[i]:inner[i+1]]:            
            m = re.match(r_element,line)
            try:
                inner_term[m.group(1)] = m.group(2)
            except:
                continue
        #add each inner object into the related-terms list
        related_list.append(inner_term)
    #add key-value pair for related terms    
    term['RelatedTerms'] = related_list
    #add each term object to the list
    sport['thesaurus'].append(term)

### Output data

In [14]:
json_file = './Output/sport.dat'#need to change extension to .dat to conform requirement
with open(json_file,'w',encoding='utf-8') as f:
    json.dump(sport,f)

## 3. Summary
Give a short summary of your work done above, such as your findings.

**Answer**: The different indentation level of XML appears to be quite helpful in finding the boundaries of objects and inner list of objects.